In [ ]:
import numpy as np
import uproot as uproot
import uproot3 as uproot3
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from tqdm.notebook import tqdm

from get_MA_trio_cov_mat import get_prediction_cv_and_variations_dataframes, get_MA_trio_cov_mat_pred, extract_trio

import pickle


In [ ]:
all_df, all_vars_df, num_unisim_variations_dic, nu_overlay_run1_pot, nu_overlay_run2_pot, nu_overlay_run3_pot, dirt_run1_pot, dirt_run2_pot, dirt_run3_pot = get_prediction_cv_and_variations_dataframes()


In [ ]:
cov_MA, pred_MA, data = get_MA_trio_cov_mat_pred(
    all_df, all_vars_df, num_unisim_variations_dic, nu_overlay_run1_pot, nu_overlay_run2_pot, nu_overlay_run3_pot, dirt_run1_pot, dirt_run2_pot, dirt_run3_pot,
    use_nuwro_fake_data = True,
    reweight_nuwro_fake_data = False,
    reweight_nuwro_M_A = np.nan
)


In [ ]:
constrained_trio, constrained_trio_cov, prior_removed_MA, prior_removed_MA_sigma = extract_trio(cov_MA, pred_MA, data)

print(prior_removed_MA)
print(prior_removed_MA_sigma)
